In [20]:
from pydantic import BaseModel, field_validator, model_validator
from typing import List, Optional
from datetime import date,datetime
import uvicorn
from fastapi import FastAPI
from fastapi.testclient import TestClient
import requests
import hashlib
from enum import Enum
import re
import json

Создайте сервис для сбора обращения абонентов на основе FastAPI.
Эндпойнт должен принимать следующие атрибуты:

фамилия – с заглавной буквы, содержит только кирилицу;
имя – с заглавной буквы, содержит только кирилицу;
дату рождения;
номер телефона;
e-mail.
Все переданные атрибуты должны валидироваться с помощью модели Pydantic.
Результат сохраняется на диске в виде json-файла, содержащего переданные атрибуты.

In [21]:
class ProblemType(str, Enum):
    NO_NETWORK = "нет доступа к сети"
    PHONE_NOT_WORKING = "не работает телефон"
    NO_EMAILS = "не приходят письма"
    
class User(BaseModel):
    last_name: str
    first_name: str
    birth_date: str
    tel_number: str
    email: str
    problem_types: List[ProblemType]
    problem_datetime: str 

    @field_validator('email')
    def valid_email(cls, value):
        if "@" not in value:
            raise ValueError("email must contain @")
        return value
    
    @field_validator('tel_number')
    def valid_telephone(cls, value):
        if not re.match(r'^((8|\+7)[\- ]?)?(\(?\d{3}\)?[\- ]?)?[\d\- ]{7,10}$', value):
            raise ValueError("Неверный формат номера телефона")
        return value
        
    @field_validator('last_name', 'first_name')
    def valid_name(cls, value):
        if not re.match(r'^[А-ЯЁЙ][а-яёй]+$', value):
            raise ValueError("Значение должно быть с заглавной буквы и содержать только кирилицу")
        return value

    @field_validator('birth_date')
    def valid_birth_date(cls, value):
        if not re.match(r'^(0?[1-9]|[12][0-9]|3[01])[\\\/\-\.](0?[1-9]|1[012])[\\\/\-\.](19[0-9]{2}|20[01][0-9]|202[0-5])$', value):
            raise ValueError("Неверный формат даты рождения")

        return value
    
    @field_validator('problem_types')
    def valid_problem_types(cls, value):
        if not value:
            raise ValueError("Должна быть указана хотя бы одна причина обращения")
        if len(value) > len(ProblemType):
            raise ValueError("Слишком много причин обращения")
        return value
    
    @field_validator('problem_datetime')
    def valid_problem_datetime(cls, value):
        if not re.match(r'^(0?[1-9]|[12][0-9]|3[01])[\\\/\-\.](0?[1-9]|1[012])[\\\/\-\.](20[0-9]{2}) ([01]?[0-9]|2[0-3]):[0-5][0-9]$', value):
            raise ValueError("Неверный формат даты и времени (должен быть ДД.ММ.ГГГГ ЧЧ:ММ)")
        normalized = re.sub(r'[\\\/\-\.]', '.', value)
        try:
            problem_dt = datetime.strptime(normalized, '%d.%m.%Y %H:%M')
            if problem_dt > datetime.now():
                raise ValueError("Дата и время обнаружения проблемы не могут быть в будущем")
        except ValueError as e:
            raise ValueError(f"Некорректная дата или время: {str(e)}")
        
        return value
    
    def generate_hash(self):
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
        data_to_hash = f"{self.email}{current_time}"
        hash_object = hashlib.sha256(data_to_hash.encode())
        return hash_object.hexdigest()


In [22]:
app = FastAPI()

@app.post("/request")
async def addRequest(request: User = None):
    try:
        filename = f"{request.generate_hash()}.txt"
        
        with open(filename, 'a', encoding='utf-8') as file:
            user_data = request.dict()
            json.dump(user_data, file, ensure_ascii=False)
            file.write('\n') 
        
        return {"status": "ok", "filename": f"{filename}"}
    except Exception as e:
        return {"status": "bad", "error": f"{type(e).__name__}: {str(e)}"}

client = TestClient(app)

In [24]:
def check_error_response(result, expected_keywords):
    if 'detail' in result:

        for error in result['detail']:
            error_msg = error['msg']
            # Проверяем содержит ли сообщение об ошибке нужные ключевые слова
            if any(keyword in error_msg.lower() for keyword in expected_keywords):
                return True
        return False
    elif result.get("status") == "bad":
        # Custom error
        error_msg = str(result.get("error", ""))
        return any(keyword in error_msg.lower() for keyword in expected_keywords)
    return False

def simple_test_1():
    """1. Простой успешный тест"""
    client = TestClient(app)
    
    response = client.post("/request", json={
        "last_name": "Иванов",
        "first_name": "Иван", 
        "birth_date": "15.05.1990",
        "tel_number": "+79123456789",
        "email": "ivan@mail.ru",
        "problem_types": ["нет доступа к сети"],
        "problem_datetime": "10.01.2024 14:30"
    })
    
    result = response.json()
    print(f"Тест 1: {result}")
    if result.get("status") == "ok":
        print("Успешный запрос")
    else:
        print("Ошибка:", result.get("error", "Неизвестная ошибка"))

def simple_test_2():
    """2. Тест с плохим email"""
    client = TestClient(app)
    
    response = client.post("/request", json={
        "last_name": "Иванов",
        "first_name": "Иван", 
        "birth_date": "15.05.1990",
        "tel_number": "+79123456789",
        "email": "bad-email",  # нет @
        "problem_types": ["нет доступа к сети"],
        "problem_datetime": "10.01.2024 14:30"
    })
    
    result = response.json()
    print(f"Тест 2: {result}")
    
    if check_error_response(result, ['email', '@']):
        print("Правильно отловил плохой email")
    else:
        print("Не отловил плохой email")

def simple_test_3():
    """3. Тест с плохим телефоном"""
    client = TestClient(app)
    
    response = client.post("/request", json={
        "last_name": "Иванов",
        "first_name": "Иван", 
        "birth_date": "15.05.1990",
        "tel_number": "123",  # неверный формат
        "email": "ivan@mail.ru",
        "problem_types": ["нет доступа к сети"],
        "problem_datetime": "10.01.2024 14:30"
    })
    
    result = response.json()
    print(f"Тест 3: {result}")
    
    if check_error_response(result, ['номер', 'телефон', 'формат']):
        print("Правильно отловил плохой телефон")
    else:
        print("Не отловил плохой телефон")

def simple_test_4():
    """4. Тест с маленькой буквой в имени"""
    client = TestClient(app)
    
    response = client.post("/request", json={
        "last_name": "Иванов",
        "first_name": "иван",  # маленькая буква
        "birth_date": "15.05.1990",
        "tel_number": "+79123456789",
        "email": "ivan@mail.ru",
        "problem_types": ["нет доступа к сети"],
        "problem_datetime": "10.01.2024 14:30"
    })
    
    result = response.json()
    print(f"Тест 4: {result}")
    
    if check_error_response(result, ['заглавной', 'кириллиц']):
        print("Правильно отловил маленькую букву")
    else:
        print("Не отловил маленькую букву")

def simple_test_5():
    """5. Тест с неправильной датой"""
    client = TestClient(app)
    
    response = client.post("/request", json={
        "last_name": "Иванов",
        "first_name": "Иван", 
        "birth_date": "1990-05-15",  # неправильный формат
        "tel_number": "+79123456789",
        "email": "ivan@mail.ru",
        "problem_types": ["нет доступа к сети"],
        "problem_datetime": "10.01.2024 14:30"
    })
    
    result = response.json()
    print(f"Тест 5: {result}")
    
    if check_error_response(result, ['даты', 'формат', 'рождения']):
        print("Правильно отловил неправильную дату")
    else:
        print("Не отловил неправильную дату")

def simple_test_6():
    """6. Тест без причин обращения"""
    client = TestClient(app)
    
    response = client.post("/request", json={
        "last_name": "Иванов",
        "first_name": "Иван", 
        "birth_date": "15.05.1990",
        "tel_number": "+79123456789",
        "email": "ivan@mail.ru",
        "problem_types": [],  # пустой список
        "problem_datetime": "10.01.2024 14:30"
    })
    
    result = response.json()
    print(f"Тест 6: {result}")
    
    if check_error_response(result, ['причин', 'обращения', 'указана']):
        print("Правильно отловил отсутствие причин обращения")
    else:
        print("Не отловил отсутствие причин обращения")

def simple_test_7():
    """7. Тест с неверным форматом даты-времени"""
    client = TestClient(app)
    
    response = client.post("/request", json={
        "last_name": "Иванов",
        "first_name": "Иван", 
        "birth_date": "15.05.1990",
        "tel_number": "+79123456789",
        "email": "ivan@mail.ru",
        "problem_types": ["нет доступа к сети"],
        "problem_datetime": "2024-01-10 14:30"  # неверный формат
    })
    
    result = response.json()
    print(f"Тест 7: {result}")
    
    if check_error_response(result, ['даты', 'времени', 'формат']):
        print("Правильно отловил неверный формат даты-времени")
    else:
        print("Не отловил неверный формат даты-времени")

if __name__ == "__main__":
    print("Запуск тестов...")
    simple_test_1()
    simple_test_2() 
    simple_test_3()
    simple_test_4()
    simple_test_5()
    simple_test_6()
    simple_test_7()

Запуск тестов...
Тест 1: {'status': 'ok', 'filename': 'c120ca5d7a65bb8575bea45a08a48f6dd1527a9744bd8757d98c4643a39c0f6d.txt'}
✅ Успешный запрос
Тест 2: {'detail': [{'type': 'value_error', 'loc': ['body', 'email'], 'msg': 'Value error, email must contain @', 'input': 'bad-email', 'ctx': {'error': {}}}]}
✅ Правильно отловил плохой email
Тест 3: {'detail': [{'type': 'value_error', 'loc': ['body', 'tel_number'], 'msg': 'Value error, Неверный формат номера телефона', 'input': '123', 'ctx': {'error': {}}}]}
✅ Правильно отловил плохой телефон
Тест 4: {'detail': [{'type': 'value_error', 'loc': ['body', 'first_name'], 'msg': 'Value error, Значение должно быть с заглавной буквы и содержать только кирилицу', 'input': 'иван', 'ctx': {'error': {}}}]}
✅ Правильно отловил маленькую букву
Тест 5: {'detail': [{'type': 'value_error', 'loc': ['body', 'birth_date'], 'msg': 'Value error, Неверный формат даты рождения', 'input': '1990-05-15', 'ctx': {'error': {}}}]}
✅ Правильно отловил неправильную дату
Тес